In [ ]:
!pip install pythonping
!pip install --pre scapy[basic]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.9 MB/s eta 0:00:00
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444349 sha256=229a507cdfaff446d0f5a4ff834b35ec000d2b268aa45c875053cc9d4e01bcd7
  Stored in directory: /root/.cache/pip/wheels/98/ea/08/164e840ab2c83b892bf8b193ce9d92d029dc3e5f2b75319953
Successfully built scapy


In [ ]:
import urllib.request as rq
import json

ip = rq.urlopen("https://api.ipify.org").read().decode("UTF-8")
ip_info = json.loads(rq.urlopen("http://ip-api.com/json/" + ip).read())

print("External IP:", ip)
print("Organization:", ip_info.get("org"))
print("Location:", ip_info.get("city"), ',', ip_info.get("country"))

External IP: 35.202.121.207
Organization: Google Cloud (us-central1)
Location: Council Bluffs , United States


In [ ]:
from pythonping import ping

target = "ftp.nz.debian.org"
ping(target, verbose=False, count=20)

Reply from 163.7.134.112, 29 bytes in 191.9ms
Reply from 163.7.134.112, 29 bytes in 191.09ms
Reply from 163.7.134.112, 29 bytes in 190.99ms
Reply from 163.7.134.112, 29 bytes in 190.95ms
Reply from 163.7.134.112, 29 bytes in 191.0ms
Reply from 163.7.134.112, 29 bytes in 191.08ms
Reply from 163.7.134.112, 29 bytes in 191.0ms
Reply from 163.7.134.112, 29 bytes in 191.05ms
Reply from 163.7.134.112, 29 bytes in 191.21ms
Reply from 163.7.134.112, 29 bytes in 190.92ms
Reply from 163.7.134.112, 29 bytes in 191.02ms
Reply from 163.7.134.112, 29 bytes in 190.99ms
Reply from 163.7.134.112, 29 bytes in 191.04ms
Reply from 163.7.134.112, 29 bytes in 191.17ms
Reply from 163.7.134.112, 29 bytes in 191.05ms
Reply from 163.7.134.112, 29 bytes in 191.08ms
Reply from 163.7.134.112, 29 bytes in 191.07ms
Reply from 163.7.134.112, 29 bytes in 191.12ms
Reply from 163.7.134.112, 29 bytes in 191.31ms
Reply from 163.7.134.112, 29 bytes in 191.06ms

Round Trip Times min/avg/max is 190.92/191.1/191.9 ms

In [ ]:
import scapy.all as sc

sc.IP().show()

sc.ICMP().show()

sc.TCP().show()

sc.UDP().show()


###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = 0
  chksum    = None
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \

###[ ICMP ]### 
  type      = echo-request
  code      = 0
  chksum    = None
  id        = 0x0
  seq       = 0x0
  unused    = ''

###[ TCP ]### 
  sport     = 20
  dport     = 80
  seq       = 0
  ack       = 0
  dataofs   = None
  reserved  = 0
  flags     = S
  window    = 8192
  chksum    = None
  urgptr    = 0
  options   = ''

###[ UDP ]### 
  sport     = 53
  dport     = 53
  len       = None
  chksum    = None



We can create a packet like this:

In [ ]:
packet = sc.IP(dst="ftp.nz.debian.org") / sc.ICMP()
packet.show()

###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = 1
  chksum    = None
  src       = 172.28.0.12
  dst       = Net("ftp.nz.debian.org/32")
  \options   \
###[ ICMP ]### 
     type      = echo-request
     code      = 0
     chksum    = None
     id        = 0x0
     seq       = 0x0
     unused    = ''



For sending the packet that we have created, we will use ```sr()```

In [ ]:
# wether we get an answer or not the two variables will be populated
ans, unans = sc.sr(packet, verbose=False) 
print("Recieved", len(ans), "packets")
print(ans.summary())
print(unans.summary())

query = ans[0][0]
answer = ans[0][1]

print(query.summary())
print(answer.summary())

Recieved 1 packets
IP / ICMP 172.28.0.12 > 163.7.134.112 echo-request 0 ==> IP / ICMP 163.7.134.112 > 172.28.0.12 echo-reply 0
None
None
IP / ICMP 172.28.0.12 > 163.7.134.112 echo-request 0
IP / ICMP 163.7.134.112 > 172.28.0.12 echo-reply 0


In [ ]:
print(query.sent_time)
print(answer.time)
rtt = answer.time - query.sent_time
print(rtt*1000)

1677156235.8036451
1677156235.9957817
192.1365261077881


# Ping

In [ ]:
def ping(dst: str, count: int) -> None:
  rtt_sum = 0; loss_count = 0; max=0; min=float("inf");
  for i in range(count):
    packet = sc.IP(dst="ftp.nz.debian.org") / sc.ICMP()
    ans, unans = sc.sr(packet, verbose=False)
    if ans is None:
      print("Request timout")
      loss_count += 1
    else:  
      rtt = 1000 * (ans[0][1].time - ans[0][0].sent_time)
      rtt_sum += rtt
      print(f"Reply from {dst} in {rtt}")
      if rtt > max:
        max = rtt
      if rtt < min:
        min = rtt
  avg =  rtt_sum / (count - loss_count)
  loss = loss_count / count
  print(f"Statistics are min={min}ms, max={max}ms, average={avg}ms, loss={loss}")


In [ ]:
ping("ftp.nz.debian.org", 5)

Reply from ftp.nz.debian.org in 191.61367416381836
Reply from ftp.nz.debian.org in 190.4599666595459
Reply from ftp.nz.debian.org in 190.57774543762207
Reply from ftp.nz.debian.org in 190.4733180999756
Reply from ftp.nz.debian.org in 190.62161445617676
Statistics are min=190.4599666595459ms, max=191.61367416381836ms, average=190.74926376342773ms, loss=0.0


# Traceroute 

In [ ]:
!apt-get install traceroute

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  traceroute
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 45.4 kB of archives.
After this operation, 152 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 traceroute amd64 1:2.1.0-2 [45.4 kB]
Fetched 45.4 kB in 0s (160 kB/s)
Selecting previously unselected package traceroute.
(Reading database ... 128126 files and directories currently installed.)
Preparing to unpack .../traceroute_1%3a2.1.0-2_amd64.deb ...
Unpacking traceroute (1:2.1.0-2) ...
Setting up traceroute (1:2.1.0-2) ...
update-alternatives: using /usr/bin/traceroute.db to provide /usr/bin/traceroute (traceroute) in auto mode
update-alternatives: using /usr/bin/traceroute6

In [ ]:
!traceroute -I ftp.nz.debian.org

traceroute to ftp.nz.debian.org (163.7.134.112), 30 hops max, 60 byte packets
 1  172.28.0.1 (172.28.0.1)  0.043 ms  0.020 ms  0.005 ms
 2  66.249.94.186 (66.249.94.186)  46.372 ms  46.357 ms  46.351 ms
 3  six.reannz.co.nz (206.81.81.203)  46.122 ms  46.120 ms  46.117 ms
 4  and33-hnl.reannz.co.nz (210.7.33.244)  93.731 ms  93.729 ms  93.725 ms
 5  and31-mgw.reannz.co.nz (210.7.33.246)  189.357 ms  189.385 ms  189.383 ms
 6  and12-nsh.reannz.co.nz (210.7.33.242)  189.222 ms  188.508 ms  188.471 ms
 7  163.7.134.120 (163.7.134.120)  190.978 ms  190.757 ms  190.729 ms
 8  163.7.134.112 (163.7.134.112)  190.194 ms  190.427 ms  190.372 ms


In [ ]:
def tracert(dst: str) -> None:
  for i in range(64):
    packet = sc.IP(dst=dst, ttl=i+1) / sc.ICMP()
    ans, unans = sc.sr(packet, verbose=False, timeout=2)
    if ans is None:
      print(f"hop {i + 1}: ***")
    else:
      rtt = 1000 * (ans[0][1].time - ans[0][0].sent_time)
      print(f"hop {i + 1}: {ans[0][1].src}, {rtt}ms")
      if not ans[0][1].type: # type 11: ttl exceeded 
        break
      

In [ ]:
tracert("ftp.nz.debian.org")

hop 1: 172.28.0.1, 0.08511543273925781ms
hop 2: 66.249.94.186, 45.93944549560547ms
hop 3: 206.81.81.203, 44.71731185913086ms
hop 4: 210.7.33.244, 93.53113174438477ms
hop 5: 210.7.33.246, 188.42029571533203ms
hop 6: 210.7.33.242, 188.51089477539062ms
hop 7: 163.7.134.120, 191.09439849853516ms
hop 8: 163.7.134.112, 190.45686721801758ms


If a server doesn't reply to ICMP, we can implement it with other protocols(TCP, UDP, ...)

## Iperf 
Iperf and SNMP were examined the second lab but since they were active measurements. I've put it here

In [1]:
!apt-get install iperf3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libiperf0 libsctp1
Suggested packages:
  lksctp-tools
The following NEW packages will be installed:
  iperf3 libiperf0 libsctp1
0 upgraded, 3 newly installed, 0 to remove and 19 not upgraded.
Need to get 94.1 kB of archives.
After this operation, 331 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libsctp1 amd64 1.0.18+dfsg-1 [7,876 B]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libiperf0 amd64 3.7-3 [72.0 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 iperf3 amd64 3.7-3 [14.2 kB]
Fetched 94.1 kB in 1s (121 kB/s)
Selecting previously unselected package libsctp1:amd64.
(Reading database ... 128221 files and directories currently installed.)
Preparing to unpack .../libsctp1_1.0.18+dfsg-1_amd64.deb ...
Unpacking libsctp1:amd64 (1.0.18+dfsg-1) ...
Selecting pr

In [16]:
# if we don't specify anything TCP will be used.
!iperf3 -c paris.testdebit.info -p 9240 --get-server-output

Connecting to host paris.testdebit.info, port 9240
[  5] local 172.28.0.12 port 56336 connected to 89.84.1.194 port 9240
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec  3.68 MBytes  30.9 Mbits/sec   15    747 KBytes       
[  5]   1.00-2.00   sec  20.0 MBytes   168 Mbits/sec    0   7.63 MBytes       
[  5]   2.00-3.00   sec  26.2 MBytes   220 Mbits/sec    0   7.63 MBytes       
[  5]   3.00-4.00   sec  26.2 MBytes   220 Mbits/sec    0   7.63 MBytes       
[  5]   4.00-5.00   sec  26.2 MBytes   220 Mbits/sec    0   7.63 MBytes       
[  5]   5.00-6.00   sec  26.2 MBytes   220 Mbits/sec    0   7.63 MBytes       
[  5]   6.00-7.00   sec  26.2 MBytes   220 Mbits/sec    0   7.63 MBytes       
[  5]   7.00-8.00   sec  26.2 MBytes   220 Mbits/sec    0   7.63 MBytes       
[  5]   8.00-9.00   sec  26.2 MBytes   220 Mbits/sec    0   7.63 MBytes       
[  5]   9.00-10.00  sec  25.0 MBytes   210 Mbits/sec    0   7.63 MBytes       
- - - - - - - - - - - - 

In [17]:
# The arg -u makes the iperf to use UDP 
# Arg -b is also used for bitrate
!iperf3 -u -c speedtest.shinternet.ch -p  5200-5209 -b 1000000000 --get-server-output

Connecting to host speedtest.shinternet.ch, port 5200
[  5] local 172.28.0.12 port 60392 connected to 88.213.191.212 port 5200
[ ID] Interval           Transfer     Bitrate         Total Datagrams
[  5]   0.00-1.00   sec   119 MBytes   999 Mbits/sec  88724  
[  5]   1.00-2.00   sec   119 MBytes  1000 Mbits/sec  88767  
[  5]   2.00-3.00   sec   119 MBytes  1.00 Gbits/sec  88806  
[  5]   3.00-4.00   sec   119 MBytes  1000 Mbits/sec  88775  
[  5]   4.00-5.00   sec   119 MBytes  1.00 Gbits/sec  88815  
[  5]   5.00-6.00   sec   119 MBytes   999 Mbits/sec  88696  
[  5]   6.00-7.00   sec   119 MBytes  1.00 Gbits/sec  88804  
[  5]   7.00-8.00   sec   119 MBytes  1.00 Gbits/sec  88825  
[  5]   8.00-9.00   sec   119 MBytes  1000 Mbits/sec  88772  
[  5]   9.00-10.00  sec   119 MBytes   999 Mbits/sec  88733  
- - - - - - - - - - - - - - - - - - - - - - - - -
[ ID] Interval           Transfer     Bitrate         Jitter    Lost/Total Datagrams
[  5]   0.00-10.00  sec  1.16 GBytes  1000 Mbits

## SNMP